In [83]:
import pandas as pd
import mysql.connector

In [84]:
# Dados de conexão com o MySQL
mysql_config = {
    'user': 'root',
    'password': '75221139',
    'host': 'localhost',
    'database': 'DW_ESTACOES_GREEN'
}

In [85]:
# Caminho para o arquivo CSV
dados_csv = 'degradação_Public.csv'

In [86]:
# Ler os dados do CSV com Pandas
df = pd.read_csv(dados_csv, skiprows= 1)

In [87]:
# Conectar ao banco de dados MySQL
cnx = mysql.connector.connect(**mysql_config)
cursor = cnx.cursor()

In [88]:
df.head()

,TIMESTAMP,RECORD,BattV,PTemp_C,Solar_Wm2,Solar_KJ,Rain_mm,WS_ms,WindDir,AirTC,RH,piranometro_dif,Tot24,Batt_Volt,Crnt_A
0,TS,RN,Volts,Deg C,W/m^2,kJ/m^2,mm,meters/second,degrees,Deg C,%,w/m2,NaN,Volts,A
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-05-13 12:18:53.81,905656,13.00102,40.11955,757.2134,45.4328,0,3.002067,81.01932,27.30967,46.93741,312.0558,0.0,13.00102,25.52767
3,2019-05-13 12:57:04.85,905695,13.01164,40.39518,719.0579,43.14346,0,5.197267,141.0386,28.02016,43.01175,313.8326,0.0,12.99511,27.10283
4,2019-05-14 06:59:53.87,945,12.58418,20.74421,73.49069,4.409441,0,0,96.35146,20.77311,80.52268,248.9822,0.0,12.5897,50.00168


In [89]:
# Remover a primeira e a segunda linha
df = df.iloc[2:] 

In [90]:
df.head()

,TIMESTAMP,RECORD,BattV,PTemp_C,Solar_Wm2,Solar_KJ,Rain_mm,WS_ms,WindDir,AirTC,RH,piranometro_dif,Tot24,Batt_Volt,Crnt_A
2,2019-05-13 12:18:53.81,905656,13.00102,40.11955,757.2134,45.4328,0,3.002067,81.01932,27.30967,46.93741,312.0558,0.0,13.00102,25.52767
3,2019-05-13 12:57:04.85,905695,13.01164,40.39518,719.0579,43.14346,0,5.197267,141.0386,28.02016,43.01175,313.8326,0.0,12.99511,27.10283
4,2019-05-14 06:59:53.87,945,12.58418,20.74421,73.49069,4.409441,0,0,96.35146,20.77311,80.52268,248.9822,0.0,12.5897,50.00168
5,2019-05-14 07:01:02.34,947,12.61719,20.98822,82.30956,4.938572,0,0,101.6905,21.07772,80.01419,252.5397,0.0,12.61719,49.00924
6,2019-05-20 15:53:30.53,10119,13.48819,34.53694,281.876,16.91256,0,2.706433,285.4698,25.69729,32.22247,320.4215,0.0,13.48268,40.99231


In [91]:
df.dtypes

TIMESTAMP           object
RECORD              object
BattV               object
PTemp_C             object
Solar_Wm2           object
Solar_KJ            object
Rain_mm             object
WS_ms               object
WindDir             object
AirTC               object
RH                  object
piranometro_dif     object
Tot24              float64
Batt_Volt           object
Crnt_A              object
dtype: object

In [92]:
# Adicionar a coluna 'nome_estacao' ao DataFrame
df['Nome_Estacao'] = 'Girassol'

In [93]:
for index, row in df.iterrows():
    # 1. Tabela TEMPO:
    datetime_obj = pd.to_datetime(row['TIMESTAMP'])
    ano = datetime_obj.year
    mes = datetime_obj.month
    dia = datetime_obj.day
    hora = datetime_obj.hour
    minuto = datetime_obj.minute

    # Verificar se o registro de tempo já existe
    cursor.execute("SELECT idTEMPO FROM DIM_TEMPO WHERE Ano = %s AND Mes = %s AND Dia = %s AND Hora = %s AND Minuto = %s", (ano, mes, dia, hora, minuto))
    result = cursor.fetchone()

    if result:
        tempo_id = result[0]
    else:
        # Inserir novo registro na tabela TEMPO
        cursor.execute("INSERT INTO DIM_TEMPO (Ano, Mes, Dia, Hora, Minuto) VALUES (%s, %s, %s, %s, %s)", (ano, mes, dia, hora, minuto))
        tempo_id = cursor.lastrowid

    # 2. Tabela DIM_ESTACAO:
    nome_estacao = row['Nome_Estacao']

    cursor.execute("SELECT idDIM_ESTACAO FROM DIM_ESTACAO WHERE nome = %s", (nome_estacao,))
    result = cursor.fetchone()

    if result:
        estacao_id = result[0]
    else:
        cursor.execute("INSERT INTO DIM_ESTACAO (nome) VALUES (%s)", (nome_estacao,))
        estacao_id = cursor.lastrowid

    # 3. Tabela DIM_VENTO:
    direcao_vento = row['WindDir']

    cursor.execute("SELECT idDIM_VENTO FROM DIM_VENTO WHERE WindDir = %s", (direcao_vento,))
    result = cursor.fetchone()

    if result:
        vento_id = result[0]
    else:
        cursor.execute("INSERT INTO DIM_VENTO (WindDir) VALUES (%s)", (direcao_vento,))
        vento_id = cursor.lastrowid

    # 4. Tabela FATO_GREEN:
    # Verificar se o registro já existe (usando a chave composta)
    cursor.execute("""
        SELECT 1 
        FROM FATO_GREEN 
        WHERE TEMPO_idTEMPO = %s 
          AND DIM_ESTACAO_idDIM_ESTACAO = %s 
          AND DIM_VENTO_idDIM_VENTO = %s
    """, (tempo_id, estacao_id, vento_id))
    result = cursor.fetchone()

    if not result:  # Inserir apenas se o registro não existir
        cursor.execute("""
            INSERT INTO FATO_GREEN (
                AirTC, Batt_Volt, BattV, Crnt_A, PTemp_C, piranometro_dif, Rain_mm, RH, Solar_KJ, Solar_Wm2, Tot24, WS_ms, TEMPO_idTEMPO, DIM_ESTACAO_idDIM_ESTACAO, DIM_VENTO_idDIM_VENTO
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['AirTC'], row['Batt_Volt'], row['BattV'], row['Crnt_A'],
            row['PTemp_C'], row['piranometro_dif'], row['Rain_mm'], row['RH'], row['Solar_KJ'], row['Solar_Wm2'], row['Tot24'], row['WS_ms'], tempo_id, estacao_id, vento_id
        ))

In [94]:
# Commitar as mudanças no banco de dados
cnx.commit()

# Fechar a conexão
cursor.close()
cnx.close()